In [5]:
search_for = 122
start_from = 0
threads = 10
thread = 4
log_freq = 10000



#in this cell, for a given lattice, we compute all forbidden nodes, 
#i.e. those nodes whose Voronoi cell is at a distance less than doubled covering radius

#An* represented in m=n+1 as sum x_i=0 with x_i\equiv to each other mod m. 
#Forbidden nodes will be recorded w.r.t the basis (-n,1,1,..) and permutations

n=5 #lattice dim
m=n+1 #space dim




R = sqrt(n*(n+1)*(n+2)/12) #covering radius

#first we generate all nodes with norm <=4R, and with norm<=2R
t=1 #.3 #can increase a bit for random from non-forbidden
short = []
vshort = []
K = ceil(t*4*R)
K2 = (t*4*R)^2
Kv = (2*R)^2

indexes = [] #arrays of integers between -K and K having the same residue modulo m 
for rem in range(m):
    res = []
    for i in range(-K,K+1):
        if i%m==rem:
            res.append(i)
    indexes.append(res)

for c in range(0,K+1):
    for coefs in mrange_iter([indexes[c%m] for i in range(m-2)]):
        p = [c]+list(coefs)+[-c-sum(coefs)]
        is_sorted = True
        for i in range(m-1):
            if p[i]<p[i+1]:
                is_sorted = False
                break
        if is_sorted:
            norm2 = sum((p[i])^2 for i in range(m))
            if norm2>0:
                if norm2 <= Kv:
                    vshort.append(p)
                if norm2 <= K2:
                    short.append(p)           

print("very short:",len(vshort))
print("short:",len(short))


from time import time
tm = time()


#defining the cone arising due to symmetry - it suffices to construct Voronoi cell in it
ineqs = [] 
for i in range(m-1):
    toadd = [0]*(m+1)
    toadd[i+1]=1
    toadd[i+2]=-1
    ineqs.append(toadd) 
    
ineqs += [[0]+[1]*m,[0]+[-1]*m] #we are in the hyperplane where all coords sum to zero

#now Voronoi cell planes
for p in vshort: #suffices to look at nodes at most 2R
    ineqs.append([sum((p[i])^2 for i in range(m))]+[-2*p[i] for i in range(m)])

V = Polyhedron(ieqs=ineqs)
    
print(time()-tm)    

### orthogonal projection affine operator onto (improper) non-empty polytope 
zerom = matrix(m)
def projp(P):
    nv = P.n_vertices()
    z = vector(P.vertices()[0])
    if nv==1:
        return zerom,z
    A = matrix([vector(P.vertices()[i])-z for i in range(1,nv)]).transpose()
    return A*A.pseudoinverse(),z


forb = []

for d in range(0,n): #dim of face
    for ff in V.faces(d):
        proj = projp(ff)
        for p in short:
            if p not in forb:
                x = vector([t/2 for t in p]) #dist from 1/2 of a node to the Voronoi cell equals 1/2 of the dist between cells at zero and at the node
                proj_pnt = proj[0]*(x-proj[1])+proj[1]
                if proj_pnt in ff.as_polyhedron():
                    dist = (x-proj_pnt).norm()
                    if dist<R:
                        forb.append(p)

print("forbidden:",len(forb))

#expand by the basis (-n,1,1,..) and permutations
def basis_coefs(x):
    y = x[0:(m-1)]
    sy = sum(y)
    return [(-sy-y[i])/m for i in range(m-1)]

### forbd = forb with possible permutations
forbd = {}
forbdv = []
for x in forb:
    for t in Permutations(list(x)):
        c = vector(basis_coefs(t))
        if str(c) not in forbd:
            forbd[str(c)]=0
            if sum(c)>=0:
                forbdv.append(c)
print("forbidden without symmetries:",len(forbd))
###





fn = 0
for x in forb:
    norm2 = sum(t^2 for t in x)
    if norm2>fn:
        fn = norm2
print("square L2 norm of forbidden:",fn)


small = 2 #max abs value of a small coefficient
sc = []
for c in mrange_iter([range(-small,small+1) for i in range(n)]):
    if sum(c)>=0:
        sc.append(vector(c))
sc.sort(key=lambda i: sum(x*x for x in i)) #abs(x) in place of x*x seems a bit slower
sc = sc[1:] #removing zero

print("small:",len(sc))

order = [(-1)^i*(1/4+i/2)-1/4 for i in range(1000)] #[0,-1,1,-2,2,...]

def check_sublattice(basis,target=Infinity): 
    A = matrix(basis) #rows
    #A = A.LLL() #benchmark to see if helps
    A = A.transpose() #columns

#     Ad = abs(A.det())
#     if Ad==0 or Ad>=target:
#         return Infinity
        
    for coefs in sc:
        if str(A*coefs) in forbd:
            return Infinity

    Ai = A.inverse()
    Ai_norms = [sum(t^2 for t in Ai[i]) for i in range(n)]
    M = []
    Mp = 1
    Mps = 1
    for i in range(n):
        mi = floor(sqrt(fn*Ai_norms[i]))
        M.append(mi)
        Mp *= (2*mi+1)
        Mps *= (2*min(small,mi)+1)
        
    if max(M)>small: #otherwise already considered
        if Mp-Mps<len(forbd): #faster using A, otherwise A_inv
            for coefs in mrange_iter([order[0:(2*M[i]+1)] for i in range(n)]):
                if sum(coefs)>=0 and max(abs(x) for x in coefs)>small:
                    if str(A*vector(coefs)) in forbd:
                        return Infinity
        else: #check if A_inv times forbidden is not in Z^n
            for x in forbdv:
                y = Ai*x
                allint = True
                for t in y:
                    if t!=floor(t):
                        allint = False
                        break
                if allint:
                    return Infinity   
    
    return 0#Ad



#random/exhaustive sublattice search, skip to the cell starting with good for search using non-forbidden generating vectors of small norm
#each type of a matrix is the numbers on the diagonal
maxd = search_for #bound for diagonal entries
mindet = search_for #minimum determinant
maxdet = search_for #maximum determinant

def all_types(m,d1,d2,n):
    res = []
    if n==1:
        for k in range(d1,min(d2,m)+1):
            res.append([k])
        return res
    for k in range(1,min(d2,m)+1):
        prev = all_types(m,ceil(d1/k),floor(d2/k),n-1)
        for x in prev:
            res.append([k]+x)
    return res

types = all_types(maxd,mindet,maxdet,n)

print("types:",len(types))

def ind_max(t): #given type, lists maxima of indexes to define entries above the diagonal
    return [t[i]^i for i in range(1,n)]


allc = 0
for t in types:
    allc += prod(ind_max(t))
print("all cases:",allc)

from random import randint
def rand_ind(t): #given type, return random indexes
    maxs = ind_max(t)
    return [randint(0,maxs[i]-1) for i in range(n-1)]

def sublat_mat(t,ind): #given type and indexes, return sublattice basis
    res = matrix(n)
    for i in range(n):
        res[i,i] = t[i]
    for i in range(1,n):
        tt = -floor(t[i]/2)
        indc = ind[i-1]
        for j in range(i):
            res[j,i] = tt + (indc % t[i])
            indc = floor(indc / t[i])
    return res

def exh_search():
    tm = time()
    bestv = Infinity
    best = []
    allcnt = 0
    for tt in types:
        print("type:",tt,prod(tt))
        cnt = 0
        maxinds = ind_max(tt)
        cM = prod(maxinds)
        print("cases of this type:",cM)
        for i in range(cM):
            allcnt += 1
            if allcnt%(log_freq*threads)==0:
                print(allcnt,(time()-tm)/3600*(allc-allcnt)/(log_freq*threads))
                tm=time()
            if allcnt>=start_from and allcnt%threads==thread:
                inds = []
                l = i
                for j in range(n-1):
                    inds.append(l%maxinds[j])
                    l=floor(l/maxinds[j])
                mat = sublat_mat(tt,inds)
                cs = check_sublattice(mat)
                if cs==0:
                    cs = prod(tt)
                    if cs<bestv:
                        print("sublattice basis:")
                        print(mat)
                        bestv = cs
                        best = [mat]
                    elif cs==bestv:
                        best.append(mat)
        print("best so far:",len(best))
    print("all cases:",allcnt)
    print("best:",len(best))
    return best





best = exh_search()




very short: 5
short: 39
0.004212141036987305
forbidden: 31
forbidden without symmetries: 1984
square L2 norm of forbidden: 264
small: 1752
types: 25
all cases: 436374755
type: [1, 1, 1, 1, 122] 122
cases of this type: 221533456
100000 21.195355075018092
200000 33.22645239445206
300000 33.54668148177567
400000 32.43725130749913
500000 34.10916434855213
600000 32.40952442237289
700000 31.98723333193962
800000 32.29269928898475
900000 27.015526781246272
1000000 18.355452612297576
1100000 31.935856129886094
1200000 31.65492055775422
1300000 31.912800869656028
1400000 33.96594659867858
1500000 32.28019953560622
1600000 31.961446494992547
1700000 32.33258727209875
1800000 29.778238013021717
1900000 14.573905505313615
2000000 30.725416699238917
2100000 30.18974469328583
2200000 30.659673096567555
2300000 31.319111332330277
2400000 30.591376829875554
2500000 29.646927159982813
2600000 29.83304931606066
2700000 29.782535163430055
2800000 13.37134846676175
2900000 29.826857618543137
3000000 29.7

29500000 31.796176509964866
29600000 30.548505829233825
29700000 30.30487612996044
29800000 28.4248844880231
29900000 28.08384508129716
30000000 12.512458955136813
30100000 28.13679776885605
30200000 28.00968026846408
30300000 28.769729486026826
30400000 30.0180014927222
30500000 31.048302655941843
30600000 31.06466458269219
30700000 16.28032972665615
30800000 30.604955799131243
30900000 31.551746121696475
31000000 30.78589925267595
31100000 30.588040474354614
31200000 15.6355980288949
31300000 30.55541967211809
31400000 30.023986045400168
31500000 29.714703884333094
31600000 28.595671196334614
31700000 26.324993559939898
31800000 15.74990264440895
31900000 23.775228320762295
32000000 26.439857461750012
32100000 32.16190578962095
32200000 29.14229628308713
32300000 30.63609330659947
32400000 31.342863663831455
32500000 15.788766883939816
32600000 31.577866030334167
32700000 33.27751843182722
32800000 31.054096114337572
32900000 31.990315926402513
33000000 14.929281225227008
33100000 36

59200000 15.50595405990588
59300000 14.367457754204391
59400000 13.838492111952231
59500000 6.812388052051419
59600000 13.512616447385957
59700000 15.158655050310916
59800000 15.924009175202581
59900000 18.914514088676555
60000000 16.731656889620655
60100000 17.562836228296796
60200000 12.99475102867195
60300000 12.955883351923996
60400000 11.44957432912456
60500000 7.498672902350714
60600000 17.130234678741875
60700000 12.242759771400925
60800000 21.460988206964604
60900000 7.68998967011103
61000000 11.751339092601677
61100000 11.874021492484777
61200000 17.444666035455253
61300000 12.83978599561371
61400000 16.078446900920362
61500000 18.167446829508055
61600000 15.305140740689053
61700000 13.762752397063593
61800000 18.495696573257806
61900000 13.604194617442971
62000000 17.204673892417432
62100000 12.714689116693389
62200000 14.21223497044967
62300000 5.971183652417958
62400000 12.146265174154777
62500000 12.379918705059303
62600000 12.234502696209976
62700000 5.172302724885123
628

88900000 26.30852596775207
89000000 29.376451651486725
89100000 27.096081927203873
89200000 24.414291783661255
89300000 22.89971511374698
89400000 23.634557216500653
89500000 24.001296973751806
89600000 22.877170028419556
89700000 19.902201555109265
89800000 15.857264962203109
89900000 14.977782346202313
90000000 18.338465452788654
90100000 13.638995164319747
90200000 22.91104067441579
90300000 23.039748506773858
90400000 24.876477177313
90500000 23.356706499618767
90600000 23.653172910501763
90700000 24.553439307615868
90800000 27.164624603411973
90900000 27.67133854592559
91000000 25.762018829452273
91100000 24.536886676749848
91200000 24.748007668120177
91300000 24.28086648022565
91400000 23.44711471219469
91500000 24.871340068976934
91600000 11.60255355689382
91700000 17.700565575659603
91800000 15.295472637736054
91900000 15.061739058910806
92000000 22.948712499810537
92100000 23.172846289944765
92200000 24.262201197942723
92300000 23.66792346470497
92400000 25.763968607183138
925

118000000 33.241981102535675
118100000 29.47071683130341
118200000 25.66121514685741
118300000 26.218277187625123
118400000 23.77727084904291
118500000 25.681648333522226
118600000 24.259241056388923
118700000 23.74218685914925
118800000 24.75773179249639
118900000 15.919040391448759
119000000 3.686735777590503
119100000 17.719265510308595
119200000 23.472863642367074
119300000 24.548849674151434
119400000 24.573441570329848
119500000 24.75612249319386
119600000 23.894380345053225
119700000 24.76076580717626
119800000 27.392558146509362
119900000 25.707376599452974
120000000 22.13626507720844
120100000 22.008509854604856
120200000 20.6957490689641
120300000 21.109948781988706
120400000 20.844701918972483
120500000 20.913809292835055
120600000 20.05886664134735
120700000 14.863545550166132
120800000 4.165886340487222
120900000 10.849233812046494
121000000 20.740714350222294
121100000 21.502391451920666
121200000 21.574826364790326
121300000 21.28882920279895
121400000 21.10258214128226


...

320200000 5.135587293469852
320300000 5.135318801875234
320400000 5.1895720106981
320500000 2.394785167273038
320600000 2.504981797603597
320700000 5.104312733032843
320800000 5.366613459367196
320900000 5.676495114629884
321000000 5.638680373552991
321100000 5.891834507126685
321200000 5.997903467235804
321300000 5.339035599885292
321400000 5.957984267229607
321500000 5.539809516960303
321600000 2.8895707444156447
321700000 4.4954385108177695
321800000 5.061824028553228
321900000 5.099552042562641
322000000 5.584846784625364
322100000 5.189507689039409
322200000 4.74646458816564
322300000 3.0017332187020576
322400000 1.6764704235139984
322500000 5.237125869416587
322600000 5.535828702450978
322700000 5.711586437880953
322800000 5.5184238719088725
322900000 5.646424363711399
323000000 5.863805697689385
323100000 5.058613199201815
323200000 6.650330806686018
323300000 5.387207958816498
323400000 2.4532794945589003
323500000 4.632644049753606
323600000 5.328069493513346
323700000 4.95897

348500000 2.9097981120978096
348600000 3.8793296101578534
348700000 3.900939210391488
348800000 4.028035162736694
348900000 3.1956235818940764
349000000 3.453754033784777
349100000 3.0871273366956413
349200000 2.6050258191943763
349300000 3.630805376355406
349400000 2.7123790859461225
349500000 3.6143188669907254
349600000 4.0037652333889895
349700000 3.9845088766796795
349800000 2.9888111207603583
349900000 3.579935574195862
350000000 3.1951299356656375
350100000 2.5961325862608686
350200000 3.6145649276791776
350300000 2.7120404758176404
350400000 3.634588314833608
350500000 4.587298600290381
350600000 3.981468712895324
350700000 2.8709037475705013
350800000 3.6738683279059727
350900000 3.572477964242488
351000000 2.436183489041759
351100000 3.68684770502962
351200000 2.5680502483213345
351300000 3.860496444955892
351400000 4.073437176870266
351500000 3.7204435313509503
351600000 3.3878192597347794
351700000 3.532025494391544
351800000 3.366992511862397
351900000 1.9772093689241543
3

377100000 0.8627095097460149
377200000 1.141931288466566
377300000 1.364402189915677
377400000 2.0756448368019393
377500000 0.9216188434348502
377600000 1.1652162266735235
377700000 1.5595647247122129
377800000 1.7460581359161835
377900000 1.15900291383335
378000000 0.868263826063868
378100000 1.3659540098753338
378200000 0.9343596519444883
378300000 1.5602992654710577
378400000 0.9868685346874563
378500000 1.1663093830357094
378600000 1.616302699198309
378700000 1.4184259758076438
378800000 1.3309823709761204
378900000 0.8218637222911352
379000000 1.4843173234859939
379100000 0.6798212297658324
379200000 1.235595286758234
379300000 1.620982042792357
379400000 1.6307280464918512
379500000 1.1637847048534056
379600000 1.2168957993390694
379700000 1.201304969076991
379800000 0.9402528438567684
379900000 1.0829525164761247
380000000 0.6800320143844486
380100000 1.0893483611234658
380200000 1.0089608054762549
380300000 1.7499591129892866
380400000 2.1561864708503156
380500000 1.29656622394

404400000 0.41230362298086615
404500000 0.5344234084058006
404600000 1.0236564717383085
404700000 0.6360416929294201
404800000 0.4617256165934006
404900000 0.37838906337168815
405000000 0.6027742731765509
405100000 0.6513530882087548
405200000 0.8122300045262277
405300000 0.313969834223946
405400000 0.5457311783398655
405500000 0.9022869801558258
405600000 0.6184416715320977
405700000 0.4958389949339165
405800000 0.37083517413976785
405900000 0.6586228815239336
406000000 0.6352312215145098
406100000 0.5734209345598816
406200000 0.31648892298143766
406300000 0.5955047889763646
406400000 0.7378394239988625
406500000 0.817926637765381
406600000 1.0206599313488272
406700000 0.6679488845013679
406800000 0.6225947248016497
406900000 0.4232662082124783
407000000 0.1331516632599632
407100000 0.13283554664325384
407200000 0.1322116840038399
407300000 0.16180123044013978
407400000 0.1331957287671632
407500000 0.13162043571815227
407600000 0.1314714936810732
407700000 0.1306902365183168
407800000

430700000 0.09613905561586222
430800000 0.09197157379189135
430900000 0.07450540443610483
431000000 0.09885460442116857
431100000 0.06659265813989772
431200000 0.09394217280952798
431300000 0.06368906429618597
431400000 0.0860825506481197
431500000 0.06813545040363404
431600000 0.07757886739797394
431700000 0.07854508338379197
431800000 0.06377253857452007
431900000 0.08091473728734255
432000000 0.06368660854903525
432100000 0.08236083867612812
432200000 0.05871991733795405
432300000 0.06818183380927642
432400000 0.05936537221590347
432500000 0.0598534867563049
432600000 0.06493719472154644
432700000 0.05166331440459357
432800000 0.06571900969946384
432900000 0.05288551456520624
433000000 0.059231362896617915
433100000 0.04581302940010031
433200000 0.05446060365464952
433300000 0.05139802846436037
433400000 0.0427904032518069
433500000 0.050473396122545
433600000 0.04057288344819347
433700000 0.04174624063798785
433800000 0.038670165815896454
433900000 0.04190889131271839
434000000 0.0